# Interpolating z-positions for plate-based imaging
__Keith Cheveralls__<br>
__November 2019__

This notebook contains scripts to facilitate the measurement and interpolation of FocusDrive positions for plate-based imaging. It operates on a list of positions generated by MicroManager's 'HCS Site Generator' plugin and exported from MicroManager's stage-position list/UI.

Its purpose is to compensate for the fact that 96-well imaging plates are tilted with respect to the focal plane of the objective and also, to some extent, not actually planar at all. 

In [ ]:
import os
import re
import sys
import json
import numpy as np
import py4j.protocol

from scipy import interpolate
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D as ax3

sys.path.insert(0, '..')
from dragonfly_automation import operations, utils
from dragonfly_automation.gateway import gateway_utils

gate, mms, mmc = gateway_utils.get_gate(env='prod', wrap=False)

# %load_ext autoreload
# %autoreload 2

### Set the path to the exported position list

In [ ]:
position_list_filepath = 'D://PR//dragonfly-automation-tests//20191106_raw_positions.pos'
with open(position_list_filepath, 'r') as file:
    position_list = json.load(file)

### Measure the FocusDrive position at a subset of wells

Here, we measure the FocusDrive (z-stage) position *after* AFC has been called at each well of a grid-like subset of wells. Detailed instructions for doing so appear below. First, in the cell below, we define the region of the plate to be imaged, and the subset of wells to visit, below. For 'normal' half-plate pipeline imaging, these parameters should not be changed.

In [ ]:
# define the region of the plate to be imaged 
# by specifying the top left and bottom right wells 
# (for half-plate imaging, these should be B2 and G9)
top_left_well_id = 'B2'
bottom_right_well_id = 'G9'

# the list of wells at which to measure FocusDrive positions
# (note that the order here is important,
# because it corresponds to the order in which they will be visited)
well_ids_to_vist = [
    'B9', 'B5', 'B2', 
    'E2', 'G2', 'G5', 'G9', 
    'E9', 'E5', 'B9',]

### Visiting each well and obtaining the FocusDrive position

In [ ]:
# run this cell without modification
visitation_manager = utils.StageVisitationManager(well_ids_to_visit, mms, mmc)

To visit each well in the list of well_ids to visit (defined above), first run the cell immediately below to move the stage to that well. Then, manually check on the microscope screen whether AFC is in range or not. If it is out of range, move the stage up (or, rarely, down) until it is back in range. Once it is in range, proceed to the next cell.

If, during this process, you forget what well you're in, please refer to the cell a few cells below about finding the well closest to the current stage position.

In [ ]:
visitation_manager.go_to_next_well()

After running the cell above and verifying that the AFC is in range, run the cell below to 'call' AFC and measure the FocusDrive position after AFC has been applied. 

In [ ]:
visitation_manager.call_afc()

Once the cell above has run, go back up to the cell before it to move the stage to the next well. Then repeat the process of manually checking that AFC is in range beflore calling the cell just above once again.

In [ ]:
# run this cell at any time to see the saved FocusDrive positions
visitation_manager.measured_focusdrive_positions

### Convenience method to find the well closest to the current stage position

Run this cell at any time to determine the well closest to the current stage position. This is useful if you forget what well you're in.

In [ ]:
utils.find_nearest_well(mmc, position_list)

### Visualize the measured positions and the interpolation

Once all of the wells in the `well_ids` list have been visited, and FocusDrive positions recorded in `measured_focusdrive_positions`, run these cells to visualize the positions and the resulting interpolation. This is meant to be a visual sanity check that the positions 'look' reasonable and are roughly planar.

In [ ]:
utils.preview_interpolation(
    visitation_manager.measured_focusdrive_positions, 
    top_left_well_id=top_left_well_id,
    bottom_right_well_id=bottom_right_well_id)

### Generate the interpolated position list

Finally, run the cell below to generate the new position list with interpolated FocusDrive positions. This new list is saved in the same directory as the original position list. 

In [ ]:
new_position_list_filepath, new_position_list = utils.interpolate_focusdrive_positions_from_all(
    position_list_filepath,
    measured_focusdrive_positions,
    top_left_well_id,
    bottom_right_well_id)

print('Interpolated position list saved to %s' % new_position_list_filepath)

### Visualize the interpolated positions
This is just a sanity check.

In [ ]:
utils.visualize_interpolation(visitation_manager.measured_focusdrive_positions, new_position_list)

### Validate the interpolation
Only run these cells after loading the interpolated position list (generated above) into the MicroManager position list.

In [ ]:
# the wells at which to check that the interpolation 'worked'
# (this list is deliberately different than the list we used to do the interpolation)
well_ids_to_vist = [
'B7', 'B3','D2',
'G3', 'G6', 'G9',
'F9', 'D9', 'D5', 'E5',
'B9']

In [ ]:
visitation_manager = StageVisitationManager(well_ids_to_visit, mms, mmc)

In [ ]:
# run this cell to visit each well; at each well, check that AFC is in range
visitation_manager.go_to_next_well()

### Crop an existing position list

Specify a list of well_ids and a number of sites per well to which to crop the list of positions. This is intended for manual redos in which the user selects, by hand, a few FOVs in a small number of wells for which no acceptable FOVs were imaged during the automated acquisition.

In [ ]:
# test filepath
position_list_filepath = '/Users/keith.cheveralls/image-data/dragonfly-automation-tests/HCS_sites_20191009_INTERPOLATED.pos'

In [ ]:
position_list_filepath = 'D://PR//dragonfly-automation-tests//'

In [ ]:
with open(position_list_filepath, 'r') as file:
    position_list = json.load(file)

In [ ]:
well_ids = ['B9', 'G9', 'A1']
num_sites_per_well = 4
site_nums = range(num_sites_per_well)

positions = position_list['POSITIONS']
positions = [p for p in positions if p['LABEL'].split('-')[0] in well_ids and int(p['LABEL'].split('_')[-1]) in site_nums]

In [ ]:
# check for well_ids that were not in the position_list
missing_well_ids = set(well_ids).difference([p['LABEL'].split('-')[0] for p in positions])
print('Warning: well_ids %s were not found in the position list' % missing_well_ids)

In [ ]:
# view the position labels explicitly
[p['LABEL'] for p in positions]

In [ ]:
# save the cropped position list
cropped_position_list = position_list.copy()
cropped_position_list['POSITIONS'] = positions

dst_filepath = position_list_filepath.replace('.pos', '_CROPPED.pos')
with open(dst_filepath, 'w') as file:
    json.dump(cropped_position_list, file)